# Enunciado

A partir do arquivo Brasileirao.xlsx que contém os resultados dos jogos do Brasileirão 2016 gerar:

* Tabela de classificação dos mandantes

* Tabela de classificação dos visitantes

* Tabela de classificação geral.

E exportar como um único excel ao final.

# Imports

In [1]:
import pandas as pd

Carregar o arquivo excel como um dataframe

In [2]:
jogos = pd.read_excel("dados/Brasileirao.xlsx")

Tratar o resultado com uma função que passa por todos as linhas e gera um dataframe. Fazer um join do dataframe original com o gerado.

In [3]:
def result(linha):
    vitoria_casa = 0
    empate = 0
    vitoria_fora = 0
    wo = 0
    gols_casa = linha["Gols Mandante"]
    gols_fora = linha["Gols Visitante"]
    if gols_casa == "WO":
        wo = 1
        gols_casa = 0
        gols_fora = 0
    elif gols_casa > gols_fora:
        vitoria_casa = 1
    elif gols_casa < gols_fora:
        vitoria_fora = 1
    else:
        empate = 1
    return pd.Series(dict(vitoria_casa=vitoria_casa,
               vitoria_fora=vitoria_fora,
               Empates=empate,
               wo=wo,
               gols_casa=gols_casa,
               gols_fora=gols_fora))

resultado = jogos.apply(lambda x: result(x),axis=1)
jogos_resultado = jogos.merge(resultado, left_index=True, right_index=True)

Agrupar a classificação mandante e visitante.

In [4]:
classificação_mandante = jogos_resultado.groupby("Mandante").agg(sum)
classificação_visitante = jogos_resultado.groupby("Visitante").agg(sum)


Renomear as colunas para fazerem sentido no contexto

In [5]:
classificação_mandante.rename(index=str, columns=
                              {"gols_casa": "Gols_Pro", 
                               "gols_fora": "Gols_Contra",
                               "vitoria_casa": "Vitórias",
                               "vitoria_fora": "Derrotas"
                              },inplace=True)
classificação_mandante.index.name = "Clube"

classificação_visitante.rename(index=str, columns=
                              {"gols_casa": "Gols_Contra", 
                               "gols_fora": "Gols_Pro",
                               "vitoria_casa": "Derrotas",
                               "vitoria_fora": "Vitórias"
                              },inplace=True)
classificação_visitante.index.name = "Clube"


Gerar colunas de pontos, jogos e saldo de gols utilizando eval que avalia a expressão pedida.

In [6]:
classificação_mandante["Pontos"] = classificação_mandante.eval("3*Vitórias+Empates") 
classificação_mandante["Jogos"] = classificação_mandante.eval("Vitórias+Empates+Derrotas+wo") 
classificação_mandante["Saldo"] = classificação_mandante.eval("Gols_Pro - Gols_Contra") 

classificação_visitante["Pontos"] = classificação_visitante.eval("3*Vitórias+Empates") 
classificação_visitante["Jogos"] = classificação_visitante.eval("Vitórias+Empates+Derrotas+wo") 
classificação_visitante["Saldo"] = classificação_visitante.eval("Gols_Pro - Gols_Contra") 


Reordenar as colunas para ficar mais bonito

In [7]:
colunas = ['Jogos','Vitórias','Empates','Derrotas','wo','Gols_Pro','Gols_Contra',"Saldo",'Pontos']
classificação_mandante = classificação_mandante[colunas]
classificação_visitante = classificação_visitante[colunas]


Gerar a classificação final.

In [8]:
classificação = classificação_mandante+classificação_visitante

Ordenar as classificações

In [9]:
classificação.sort_values(["Pontos","Vitórias","Saldo","Gols_Pro"],ascending=False,inplace=True)
classificação_mandante.sort_values(["Pontos","Vitórias","Saldo","Gols_Pro"],ascending=False,inplace=True)
classificação_visitante.sort_values(["Pontos","Vitórias","Saldo","Gols_Pro"],ascending=False,inplace=True)


Mostrar as classificações

# Classificação Mandante

In [10]:
classificação_mandante

,Jogos,Vitórias,Empates,Derrotas,wo,Gols_Pro,Gols_Contra,Saldo,Pontos
Clube,,,,,,,,,
Atletico-PR,19,15,3,1,0,27,6,21,48
Santos,19,15,2,2,0,36,11,25,47
Palmeiras,19,14,4,1,0,35,13,22,46
Atletico-MG,19,13,3,3,0,39,20,19,42
Ponte Preta,19,13,2,4,0,33,18,15,41
Flamengo RJ,19,12,5,2,0,31,16,15,41
Gremio,19,11,4,4,0,22,11,11,37
Corinthians,19,10,7,2,0,27,11,16,37
Sport Recife,19,10,5,4,0,33,22,11,35


# Classificação Visitante

In [11]:
classificação_visitante

,Jogos,Vitórias,Empates,Derrotas,wo,Gols_Pro,Gols_Contra,Saldo,Pontos
Clube,,,,,,,,,
Palmeiras,19,10,4,5,0,27,19,8,34
Flamengo RJ,19,8,6,5,0,21,19,2,30
Santos,19,7,3,9,0,23,24,-1,24
Botafogo RJ,19,7,3,9,0,16,22,-6,24
Cruzeiro,19,7,2,10,0,19,23,-4,23
Chapecoense-SC,19,5,7,7,0,24,32,-8,22
Fluminense,19,5,5,9,0,15,21,-6,20
Sao Paulo,19,5,5,9,0,15,23,-8,20
Atletico-MG,19,4,8,6,1,22,30,-8,20


# Classificação Geral

In [12]:
classificação

,Jogos,Vitórias,Empates,Derrotas,wo,Gols_Pro,Gols_Contra,Saldo,Pontos
Clube,,,,,,,,,
Palmeiras,38,24,8,6,0,62,32,30,80
Santos,38,22,5,11,0,59,35,24,71
Flamengo RJ,38,20,11,7,0,52,35,17,71
Atletico-MG,38,17,11,9,1,61,50,11,62
Botafogo RJ,38,17,8,13,0,43,39,4,59
Atletico-PR,38,17,6,15,0,38,32,6,57
Corinthians,38,15,10,13,0,48,42,6,55
Ponte Preta,38,15,8,15,0,48,52,-4,53
Gremio,38,14,11,13,0,41,44,-3,53


In [13]:
writer = pd.ExcelWriter('Tabelas Gabarito/BR2016_Pythonico.xlsx')
classificação_mandante.to_excel(writer,'Classificação Mandante')
classificação_visitante.to_excel(writer,'Classificação Visitante')
classificação.to_excel(writer,'Classificação Geral')
writer.close()